In [10]:
from pathlib import Path
import duckdb
import pandas as pd
# Define the path to the Parquet file
crs_file = Path("~/Downloads") / "CRS.parquet"
dac1_file = Path("~/Downloads") / "Table1_Data.csv"

pd.set_option('display.max_rows', 100)  # Set a specific number of rows
pd.set_option('display.width', 1000)       # Adjust overall display width
pd.set_option('display.max_colwidth', 500)


data_save_path = Path("../") 

DAC_COUNTRIES = [
  "Australia", "Austria", "Belgium", "Canada", "Czechia", "Denmark", "Finland", "France", "Germany", "Greece", "Hungary",
  "Iceland", "Ireland", "Italy", "Japan", "Korea", "Luxembourg", "Netherlands", "New Zealand", "Norway", "Poland",
  "Portugal", "Slovak Republic", "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States", "Estonia", "Lithuania"
]

## ONE Campaign

In [ ]:
# from oda_data import ODAData, set_data_path

# # set the path to the folder where the data should be stored
# set_data_path("data")

# # create object, specifying key details of the desired output
# data = ODAData(years=range(2018,2023), currency="EUR", prices="constant", base_year=2021)

# # load the desired indicators
# data.load_indicator(indicators = ["imputed_multi_flow_disbursement_gross", "total_oda_ge"])

# # get the data
# df = data.get_data()

# print(df.head(6))

## CRS DATA

In [3]:
with duckdb.connect() as conn:

    cols_query = f"""
    SELECT * FROM '{crs_file.as_posix()}'
    LIMIT 1
    """
    result = conn.execute(cols_query)

    # Get the column names from the description attribute
    column_names = [desc[0] for desc in result.description]

column_names

['year',
 'donor_code',
 'de_donorcode',
 'donor_name',
 'agency_code',
 'agency_name',
 'crs_id',
 'project_number',
 'initial_report',
 'recipient_code',
 'de_recipientcode',
 'recipient_name',
 'region_code',
 'de_regioncode',
 'region_name',
 'incomegroup_code',
 'de_incomegroup_code',
 'incomegroup_name',
 'flow_code',
 'flow_name',
 'bi_multi',
 'category',
 'finance_t',
 'aid_t',
 'usd_commitment',
 'usd_disbursement',
 'usd_received',
 'usd_commitment_defl',
 'usd_disbursement_defl',
 'usd_received_defl',
 'usd_adjustment',
 'usd_adjustment_defl',
 'usd_amount_untied',
 'usd_amount_partial_tied',
 'usd_amount_tied',
 'usd_amount_untied_defl',
 'usd_amount_partial_tied_defl',
 'usd_amounttied_defl',
 'usd_irtc',
 'usd_expert_commitment',
 'usd_expert_extended',
 'usd_export_credit',
 'currency_code',
 'commitment_national',
 'disbursement_national',
 'grant_equiv',
 'usd_grant_equiv',
 'short_description',
 'project_title',
 'purpose_code',
 'purpose_name',
 'sector_code',
 'sec

# DAC1 DATA

In [4]:
with duckdb.connect() as conn:

    cols_query = f"""
    SELECT * FROM '{dac1_file.as_posix()}'
    LIMIT 1
    """
    result = conn.execute(cols_query)

    # Get the column names from the description attribute
    column_names = [desc[0] for desc in result.description]

column_names

['DONOR',
 'Donor_1',
 'PART',
 'Part_1',
 'AIDTYPE',
 'Aid type',
 'FLOWS',
 'Fund flows',
 'AMOUNTTYPE',
 'Amount type',
 'TIME',
 'Year',
 'Value',
 'Flags']

## 2A All Donor ODA Trend

R code for 2A
TotalODAranking_previousyear <- DAC1data %>%
  filter(
    aid_type == "Official Development Assistance, grant equivalent measure",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year - 1
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year - 1,
    Donor = donor_name,
    `Total ODA` = Value / 1000,
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )

TotalODAranking_endyear <- DAC1data %>%
  filter(
    aid_type == "Official Development Assistance, grant equivalent measure",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year,
    Donor = donor_name,
    `Total ODA` = Value / 1000,
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )
TotalODAranking <- rbind(TotalODAranking_endyear, TotalODAranking_previousyear)
TotalODAranking <- TotalODAranking[order(TotalODAranking$Year), ]

In [23]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2022 AND 2023
        AND "Amount type" = 'Current Prices (USD millions)'
        AND "Fund flows" = 'Grant equivalents'
        AND "Donor_1" IN {tuple(DAC_COUNTRIES)}
        AND "Aid type" = 'Official Development Assistance, grant equivalent measure'
    ), 

    ranked AS (  
        SELECT
            donor,
            year,
            value / 1000 AS "Total ODA", 
            row_number() OVER (PARTITION BY year ORDER BY value DESC) AS rn
        FROM base
    )

    SELECT
        year AS "Year",
        donor AS "Donor",
        round("Total ODA", 4) AS "Total ODA", 
        CASE 
            WHEN rn::TEXT LIKE '%1' AND rn != 11 THEN rn || 'st'
            WHEN rn::TEXT LIKE '%2' AND rn != 12 THEN rn || 'nd'
            WHEN rn::TEXT LIKE '%3' AND rn != 13THEN rn || 'rd'
            ELSE rn || 'th'
        END AS "Rank"
    FROM ranked
    ORDER BY year, rn
    """
    result = conn.execute(query).fetchdf()

result.to_csv(data_save_path / "DT_update" / "alldonor_2A_ODATrend.csv", index=False)

## 2B ODA GNI Ranking

ODAGNIranking_previousyear <- DAC1data %>%
  filter(
    aid_type == "ODA grant equivalent as percent of GNI",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year - 1
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year - 1,
    Donor = donor_name,
    `ODA as % GNI` = paste0(round(Value, 3), "%"),
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )

ODAGNIranking_endyear <- DAC1data %>%
  filter(
    aid_type == "ODA grant equivalent as percent of GNI",
    amount_type == "Current Prices",
    donor_name %in% DACcountries,
    flow_type == "Grant equivalents",
    Year == end_year
  ) %>%
  arrange(desc(Value)) %>%
  select("donor_name", "Value") %>%
  summarise(
    Year = end_year,
    Donor = donor_name,
    `ODA as % GNI` = paste0(round(Value, 3), "%"),
    Rank = paste0(rank(-Value, ties.method = "first"), c("st", "nd", "rd", rep("th", 17)))
  )

ODAGNIranking <- rbind(ODAGNIranking_endyear, ODAGNIranking_previousyear)
ODAGNIranking <- ODAGNIranking[order(ODAGNIranking$Year), ]
print(ODAGNIranking)

In [24]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2022 AND 2023
        AND "Amount type" = 'Current Prices (USD millions)'
        AND "Fund flows" = 'Grant equivalents'
        AND "Donor_1" IN {tuple(DAC_COUNTRIES)}
        AND "Aid type" = 'ODA grant equivalent as percent of GNI'
    ), 

    ranked AS (  
        SELECT
            donor,
            year,
            round(value, 2) || '%' AS "ODA as % GNI", 
            row_number() OVER (PARTITION BY year ORDER BY value DESC) AS rn
        FROM base
    )

    SELECT
        year AS "Year",
        donor AS "Donor",
        "ODA as % GNI", 
        CASE 
            WHEN rn::TEXT LIKE '%1' AND rn != 11 THEN rn || 'st'
            WHEN rn::TEXT LIKE '%2' AND rn != 12 THEN rn || 'nd'
            WHEN rn::TEXT LIKE '%3' AND rn != 13THEN rn || 'rd'
            ELSE rn || 'th'
        END AS "Rank"
    FROM ranked
    ORDER BY year, rn
    """
    result = conn.execute(query).fetchdf()

#TODO: Slight rounding issues here. 
result.to_csv(data_save_path / "DT_update" / "alldonor_2B_ODAGNIranking.csv", index=False)

## 3A TOTAL ODA Over Time
CUT

## 3B Refugee

R code: 
donor_refugee_data <- DAC1data %>%
    filter(
      Year >= end_year - 4,
      donor_name == current_donor,
      amount_type == "Constant Prices",
      flow_type == "Net Disbursements",
      aid_type %in% c(
        "I.A.8.2. Refugees in donor countries",
        "I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)",
        "I.B.1.2. EU institutions",
        "I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)"
      )
    ) %>%
    select(aid_type, Year, Value) %>%
    group_by(Year) %>%
    summarise(
      `ODA for Development Priorities` = sum(Value * case_when(
        aid_type %in% c("I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)") ~ 1,
        aid_type %in% c("I.A.8.2. Refugees in donor countries") ~ -1,
        aid_type %in% c("I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)") ~ 1,
        aid_type %in% c("I.B.1.2. EU institutions") ~ -1,
        TRUE ~ 0
      )),
      `Contributions to EUI` = sum(Value * case_when(
        aid_type %in% c("I.B.1.2. EU institutions") ~ 1,
        TRUE ~ 0
      )),
      `In-donor Refugee Costs` = sum(Value * case_when(
        aid_type %in% c("I.A.8.2. Refugees in donor countries") ~ 1,
        TRUE ~ 0
      ))
    )


In [30]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2019 AND 2023
        AND "Amount type" = 'Constant Prices (2022 USD millions)'
        AND "Fund flows" = 'Net Disbursements'
        AND "Aid type" IN (
            'I.A.8.2. Refugees in donor countries',
            'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)',
            'I.B.1.2. EU institutions',
            'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)'
        )
    ), 

    filtered AS (
    SELECT
        donor,
        year,  
        SUM(value * 
        CASE 
            WHEN aid_type = 'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)' THEN 1
            WHEN aid_type = 'I.A.8.2. Refugees in donor countries' THEN -1
            WHEN aid_type = 'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)' THEN 1
            WHEN aid_type = 'I.B.1.2. EU institutions' THEN -1
            ELSE 0
        END
        ) AS "ODA for Development Priorities", 
        SUM(value *
        CASE 
            WHEN aid_type = 'I.B.1.2. EU institutions' THEN 1
            ELSE 0
        END
        ) AS "Contributions to EUI", 
        SUM(value *
        CASE
            WHEN aid_type = 'I.A.8.2. Refugees in donor countries' THEN 1
            ELSE 0
        END
        ) AS "In-donor Refugee Costs"
    FROM base
    GROUP BY donor, year

    )
    SELECT
        donor,
        year AS "Year",
        "ODA for Development Priorities",
        "Contributions to EUI",
        "In-donor Refugee Costs"
    FROM filtered f
    ORDER BY year, donor
    """
    result = conn.execute(query).fetchdf()


for donor in DAC_COUNTRIES:
    if donor in ['Austria', 'Belgium', 'Denmark', 'Finland', 'Ireland', 'Luxembourg', 'Switzerland']:
        save_path = data_save_path / "OP" / f"{donor}_3B_refugee.csv"
    else: 
        save_path = data_save_path / "DT_update" / f"{donor}_3B_refugee.csv"
    
    data = result[result["donor"] == donor][['Year', 'ODA for Development Priorities', 'Contributions to EUI', 'In-donor Refugee Costs']]
    data.to_csv(save_path, index=False)
    
# result[result["donor"] == "Austria"][['Year', 'ODA for Development Priorities', 'Contributions to EUI', 'In-donor Refugee Costs']].to_csv("Austria_3B_refugee.csv", index=False)

## 4 BI-MULTI

R code for Bi-multi
for (current_donor in specified_countries) {
  donor_multi_bi_data <- DAC1data %>% 
    filter(Year >= end_year_dac - 4) %>%
    filter(
      Year <= end_year_dac,
      donor_name == current_donor,
      amount_type == "Constant Prices",
      flow_type == "Gross Disbursements",
      aid_type %in% c(
        "I. Official Development Assistance (ODA) (I.A + I.B)",
        "I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)",
        "I.A. Memo: ODA channelled through multilateral organisations",
        "I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)")) %>%
    select(aid_type, Year, Value) %>%
    summarise(
      `Year` = Year[aid_type %in% c("I.A. Memo: ODA channelled through multilateral organisations")],
      `Bilateral funding` = Value[aid_type %in% c("I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)")] - Value[aid_type %in% c("I.A. Memo: ODA channelled through multilateral organisations")],
      `Bilateral as earmarked funding through multilaterals` = Value[aid_type %in% c("I.A. Memo: ODA channelled through multilateral organisations")],
      `Multilateral as core contributions to organizations` = Value[aid_type %in% c("I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)")],
      `Total ODA` = Value[aid_type %in% c("I. Official Development Assistance (ODA) (I.A + I.B)")]
    ) %>%
    mutate(
      `Bilateral` = paste0(round(`Bilateral funding` / `Total ODA` * 100), "%"),
      `Earmarked` = paste0(round(`Bilateral as earmarked funding through multilaterals` / `Total ODA` * 100), "%"),
      `Multilateral` = paste0(round(`Multilateral as core contributions to organizations` / `Total ODA` * 100), "%")
    ) %>%
    select(`Year`, `Bilateral funding`, `Bilateral as earmarked funding through multilaterals`, `Multilateral as core contributions to organizations`, `Bilateral`, `Earmarked`, `Multilateral`)
  #print(donor_multi_bi_data)

In [32]:
with duckdb.connect() as conn:
    query = f"""
    WITH base AS (
        SELECT 
            "Donor_1" AS donor,
            "Year" AS year,
            "Aid type" AS aid_type,
            "VALUE" AS value
        FROM '{dac1_file.as_posix()}'
        WHERE 1=1
        AND year BETWEEN 2019 AND 2023
        AND "Amount type" = 'Constant Prices (2022 USD millions)'
        AND "Fund flows" = 'Gross Disbursements'
        AND "Aid type" IN (
            'I. Official Development Assistance (ODA) (I.A + I.B)',
            'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)',
            'I.A. Memo: ODA channelled through multilateral organisations',
            'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)'
        )
    ), 

    filtered AS (  
        SELECT
            donor,
            year AS "Year",
            SUM(CASE WHEN aid_type = 'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)' THEN "value" END) -
            SUM(CASE WHEN aid_type = 'I.A. Memo: ODA channelled through multilateral organisations' THEN "value" END) AS "Bilateral funding",
            SUM(CASE WHEN aid_type = 'I.A. Memo: ODA channelled through multilateral organisations' THEN "value" END) AS "Bilateral as earmarked funding through multilaterals",
            SUM(CASE WHEN aid_type = 'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)' THEN "value" END) AS "Multilateral as core contributions to organizations",
            SUM(CASE WHEN aid_type = 'I. Official Development Assistance (ODA) (I.A + I.B)' THEN "value" END) AS "Total ODA"
        FROM base
        GROUP BY 1,2
    )
    SELECT
        donor,
        "Year",
        "Total ODA",
        coalesce("Bilateral funding", 0) AS "Bilateral funding",
        coalesce("Bilateral as earmarked funding through multilaterals", 0) AS "Bilateral as earmarked funding through multilaterals",
        coalesce("Multilateral as core contributions to organizations", 0) AS "Multilateral as core contributions to organizations", 
        round( 100 * coalesce("Bilateral funding", 0) / "Total ODA")::INT || '%'  AS "Bilateral",
        round( 100 * coalesce("Bilateral as earmarked funding through multilaterals", 0) / "Total ODA")::INT || '%' AS "Earmarked",
        round( 100 * coalesce("Multilateral as core contributions to organizations", 0) / "Total ODA")::INT || '%' AS "Multilateral"
    FROM filtered
    ORDER BY "Year"
    """
    result = conn.execute(query).fetchdf()


for donor in DAC_COUNTRIES:
    if donor in ['Austria', 'Belgium', 'Denmark', 'Finland', 'Ireland', 'Luxembourg', 'Switzerland']:
        save_path = data_save_path / "OP" / f"{donor}_4_bimulti.csv"
    else: 
        save_path = data_save_path / "DT_update" / f"{donor}_4_bimulti.csv"
    
    data = result[result["donor"] == donor][["Year",
                                             "Bilateral funding",
                                             "Bilateral as earmarked funding through multilaterals",
                                             "Multilateral as core contributions to organizations",
                                             "Bilateral",
                                             "Earmarked",
                                             "Multilateral"]]
    data.to_csv(save_path, index=False)


In [39]:
with duckdb.connect() as conn:
    # Run a query on the Parquet file
    query = f"""
    SELECT DISTINCT
        "Aid type"
    FROM '{dac1_file.as_posix()}'
    WHERE "Year" = 2022
    AND "Amount type" = 'Constant Prices (2022 USD millions)'
        AND "Fund flows" = 'Gross Disbursements'
        AND "Aid type" IN (
            'I. Official Development Assistance (ODA) (I.A + I.B)',
            'I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)',
            'I.A. Memo: ODA channelled through multilateral organisations',
            'I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)'
        )
    AND "Donor_1" = 'Austria'
    """
    result = conn.execute(query).fetchdf()

print(result)

                                                                                             Aid type
0  I.B. Multilateral Official Development Assistance (capital subscriptions are included with grants)
1                                                I. Official Development Assistance (ODA) (I.A + I.B)
2                                        I.A. Memo: ODA channelled through multilateral organisations
3               I.A. Bilateral Official Development Assistance by types of aid (1+2+3+4+5+6+7+8+9+10)
